In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Lakers 2019-2020

In [18]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/NBA Data/2019-2020 NBA Play-By-Play Data.csv" ,low_memory = False)

In [24]:
# preprocessing

data = data.loc[(data['points'] != None)]
data = data.loc[(data['points'] >= 0)]
data = data.loc[(data['points'] <= 3)]
data['Made Basket'] = np.where(data['points'] > 0, 1, 0)
data['Shot Attempted'] = 1
data['converted_y_2'] = np.where(data['converted_y'] > 50, 100 - data['converted_y'], data['converted_y'])

In [36]:
lakers = data.loc[data['GameFile'].str.contains("LAL")]
lakers['LAL'] = np.where(lakers['team'] == 'LAL', 'Lakers', 'Opponent')
lakers['pointspm'] = np.where(lakers['team'] == 'LAL', lakers['points'], lakers['points']*-1)
lakers['players_on_court'] = lakers['a1'] + "_" + lakers['a2'] + "_" + lakers['a3'] + "_" + lakers['a4']+ "_" + lakers['a5']+ "_" + lakers['h1']+ "_" + lakers['h2']+ "_" + lakers['h3']+ "_" + lakers['a4']+ "_" + lakers['h5']

data = lakers
team_players = ['Alex Caruso', 'Danny Green', 'Talen Horton-Tucker', 'Kyle Kuzma',
                'JaVale McGee', 'Markieff Morris','Rajon Rondo','Avery Bradley',
                'Kentavious Caldwell-Pope', 'Quinn Cook', 'Dion Waiters']
on_court = ['LeBron James', 'Anthony Davis']
players_plus_minus = {'on_court': {}, 'off_court': {}}

def check_players_on_court(players_on_court, condition_players):
    return all(player in players_on_court for player in condition_players)

for index, row in data.iterrows():
    game_id = row['game_id']
    points_scored = row['points']

    if points_scored == 0 or ('LeBron James' not in row.values and 'Anthony Davis' not in row.values):
        continue

    players_on_court = [row['a1'], row['a2'], row['a3'], row['a4'], row['a5'], row['h1'], row['h2'], row['h3'], row['h4'], row['h5']]

    check_pl = check_players_on_court(players_on_court, on_court)

    for player in team_players:
        if player in players_on_court:
            key = 'on_court' if check_pl else 'off_court'
            if (game_id, player) not in players_plus_minus[key]:
                players_plus_minus[key][(game_id, player)] = points_scored if player in players_on_court else -points_scored
            else:
                players_plus_minus[key][(game_id, player)] += points_scored if player in players_on_court else -points_scored

on_court_df = pd.DataFrame(list(players_plus_minus['on_court'].items()), columns=['Game_Player', 'PlusMinusOnCourt'])
off_court_df = pd.DataFrame(list(players_plus_minus['off_court'].items()), columns=['Game_Player', 'PlusMinusOffCourt'])

on_court_df[['GameID', 'Player']] = pd.DataFrame(on_court_df['Game_Player'].tolist(), index=on_court_df.index)
off_court_df[['GameID', 'Player']] = pd.DataFrame(off_court_df['Game_Player'].tolist(), index=off_court_df.index)

on_court_df.drop('Game_Player', axis=1, inplace=True)
off_court_df.drop('Game_Player', axis=1, inplace=True)

merged_df = pd.merge(on_court_df, off_court_df, on=['GameID', 'Player'], how='outer').fillna(0)
print(merged_df.head())


   PlusMinusOnCourt         GameID                    Player  \
0             158.0  ="0021900522"               Danny Green   
1              76.0  ="0021900522"              JaVale McGee   
2             139.0  ="0021900522"             Avery Bradley   
3              19.0  ="0021900522"                Kyle Kuzma   
4              59.0  ="0021900522"  Kentavious Caldwell-Pope   

   PlusMinusOffCourt  
0               11.0  
1                0.0  
2               13.0  
3               59.0  
4               51.0  


In [37]:
avg_plus_minus_updated = {
    'Alex Caruso': {
        'on_court': merged_df[(merged_df['Player'] == 'Alex Caruso')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Alex Caruso')]['PlusMinusOffCourt'].mean()
    },
    'Danny Green': {
        'on_court': merged_df[(merged_df['Player'] == 'Danny Green')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Danny Green')]['PlusMinusOffCourt'].mean()
    },
    'Talen Horton-Tucker': {
        'on_court': merged_df[(merged_df['Player'] == 'Talen Horton-Tucker')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Talen Horton-Tucker')]['PlusMinusOffCourt'].mean()
    },
    'Kyle Kuzma': {
        'on_court': merged_df[(merged_df['Player'] == 'Kyle Kuzma')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Kyle Kuzma')]['PlusMinusOffCourt'].mean()
    },
    'JaVale McGee': {
        'on_court': merged_df[(merged_df['Player'] == 'JaVale McGee')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'JaVale McGee')]['PlusMinusOffCourt'].mean()
    },
    'Markieff Morris': {
        'on_court': merged_df[(merged_df['Player'] == 'Markieff Morris')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Markieff Morris')]['PlusMinusOffCourt'].mean()
    },
    'Rajon Rondo': {
        'on_court': merged_df[(merged_df['Player'] == 'Rajon Rondo')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Rajon Rondo')]['PlusMinusOffCourt'].mean()
    },
    'Avery Bradley': {
        'on_court': merged_df[(merged_df['Player'] == 'Avery Bradley')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Avery Bradley')]['PlusMinusOffCourt'].mean()
    },
    'Kentavious Caldwell-Pope': {
        'on_court': merged_df[(merged_df['Player'] == 'Kentavious Caldwell-Pope')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Kentavious Caldwell-Pope')]['PlusMinusOffCourt'].mean()
    },
    'Quinn Cook': {
        'on_court': merged_df[(merged_df['Player'] == 'Quinn Cook')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Quinn Cook')]['PlusMinusOffCourt'].mean()
    },
    'Dion Waiters': {
        'on_court': merged_df[(merged_df['Player'] == 'Dion Waiters')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Dion Waiters')]['PlusMinusOffCourt'].mean()
    }
}

print("Average Plus-Minus With Stars On and Off the Court:")
for player, stats in avg_plus_minus_updated.items():
    print(f"\n{player}:")
    print(f"  On Court: {stats['on_court']:.2f}")
    print(f"  Off Court: {stats['off_court']:.2f}")


Average Plus-Minus With Stars On and Off the Court:

Alex Caruso:
  On Court: 26.14
  Off Court: 52.41

Danny Green:
  On Court: 77.12
  Off Court: 33.82

Talen Horton-Tucker:
  On Court: 0.00
  Off Court: 34.17

Kyle Kuzma:
  On Court: 25.96
  Off Court: 73.40

JaVale McGee:
  On Court: 51.70
  Off Court: 15.20

Markieff Morris:
  On Court: 15.40
  Off Court: 48.94

Rajon Rondo:
  On Court: 19.83
  Off Court: 69.67

Avery Bradley:
  On Court: 67.52
  Off Court: 39.40

Kentavious Caldwell-Pope:
  On Court: 58.31
  Off Court: 54.17

Quinn Cook:
  On Court: 6.07
  Off Court: 41.79

Dion Waiters:
  On Court: 9.18
  Off Court: 46.36


### Lakers 2018-2019

In [41]:
data = pd.read_csv("/content/drive/MyDrive/Datasets/NBA Data/2018-2019 NBA Play-By-Play Data.csv" ,low_memory = False)
data = data.loc[(data['points'] != None)]
data = data.loc[(data['points'] >= 0)]
data = data.loc[(data['points'] <= 3)]
data['Made Basket'] = np.where(data['points'] > 0, 1, 0)
data['Shot Attempted'] = 1

In [44]:
data.columns

Index(['Unnamed: 0', 'game_id', 'data_set', 'date', 'a1', 'a2', 'a3', 'a4',
       'a5', 'h1', 'h2', 'h3', 'h4', 'h5', 'period', 'away_score',
       'home_score', 'remaining_time', 'elapsed', 'play_length', 'play_id',
       'team', 'event_type', 'assist', 'away', 'home', 'block', 'entered',
       'left', 'num', 'opponent', 'outof', 'player', 'points', 'possession',
       'reason', 'result', 'steal', 'type', 'shot_distance', 'original_x',
       'original_y', 'converted_x', 'converted_y', 'description', 'GameFile',
       'Made Basket', 'Shot Attempted', 'LAL', 'pointspm', 'players_on_court'],
      dtype='object')

In [54]:
lakers = data.loc[data['GameFile'].str.contains("LAL")]
lakers['LAL'] = np.where(lakers['team'] == 'LAL', 'Lakers', 'Opponent')
lakers['pointspm'] = np.where(lakers['team'] == 'LAL', lakers['points'], lakers['points']*-1)
lakers['players_on_court'] = lakers['a1'] + "_" + lakers['a2'] + "_" + lakers['a3'] + "_" + lakers['a4']+ "_" + lakers['a5']+ "_" + lakers['h1']+ "_" + lakers['h2']+ "_" + lakers['h3']+ "_" + lakers['a4']+ "_" + lakers['h5']

#data = lakers
team_players = ['Lonzo Ball', 'Michael Beasley', 'Reggie Bullock', 'Kentavious Caldwell-Pope', 'Alex Caruso', 'Josh Hart','Brandon Ingram','Kyle Kuzma','Rajon Rondo']
on_court = ['LeBron James']
players_plus_minus = {'on_court': {}, 'off_court': {}}

def check_players_on_court(players_on_court, condition_players):
    return all(player in players_on_court for player in condition_players)

for index, row in data.iterrows():
    game_id = row['game_id']
    points_scored = row['points']
    if points_scored == 0 or ('LeBron James' not in row.values):
        continue
    players_on_court = [row['a1'], row['a2'], row['a3'], row['a4'], row['a5'], row['h1'], row['h2'], row['h3'], row['h4'], row['h5']]
    check_pl = check_players_on_court(players_on_court, on_court)
    for player in team_players:
        if player in players_on_court:
            key = 'on_court' if check_pl else 'off_court'
            if (game_id, player) not in players_plus_minus[key]:
                players_plus_minus[key][(game_id, player)] = points_scored
            else:
                players_plus_minus[key][(game_id, player)] += points_scored
        else:
            key = 'off_court'
            if (game_id, player) not in players_plus_minus[key]:
                players_plus_minus[key][(game_id, player)] = points_scored
            else:
                players_plus_minus[key][(game_id, player)] += points_scored

on_court_df = pd.DataFrame([(game_id, player, plus_minus) for (game_id, player), plus_minus in players_plus_minus['on_court'].items()], columns=['GameID', 'Player', 'PlusMinusOnCourt'])
off_court_df = pd.DataFrame([(game_id, player, plus_minus) for (game_id, player), plus_minus in players_plus_minus['off_court'].items()], columns=['GameID', 'Player', 'PlusMinusOffCourt'])

merged_df = pd.merge(on_court_df, off_court_df, on=['GameID', 'Player'], how='outer').fillna(0)

print(merged_df.head())

avg_plus_minus_updated = {
    'Lonzo Ball': {
        'on_court': merged_df[(merged_df['Player'] == 'Lonzo Ball')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Lonzo Ball')]['PlusMinusOffCourt'].mean()
    },
    'Michael Beasley': {
        'on_court': merged_df[(merged_df['Player'] == 'Michael Beasley')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Michael Beasley')]['PlusMinusOffCourt'].mean()
    },
    'Reggie Bullock': {
        'on_court': merged_df[(merged_df['Player'] == 'Reggie Bullock')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Reggie Bullock')]['PlusMinusOffCourt'].mean()
    },
    'Kentavious Caldwell-Pope': {
        'on_court': merged_df[(merged_df['Player'] == 'Kentavious Caldwell-Pope')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Kentavious Caldwell-Pope')]['PlusMinusOffCourt'].mean()
    },
    'Alex Caruso': {
        'on_court': merged_df[(merged_df['Player'] == 'Alex Caruso')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Alex Caruso')]['PlusMinusOffCourt'].mean()
    },
    'Josh Hart': {
        'on_court': merged_df[(merged_df['Player'] == 'Josh Hart')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Josh Hart')]['PlusMinusOffCourt'].mean()
    },
    'Brandon Ingram': {
        'on_court': merged_df[(merged_df['Player'] == 'Brandon Ingram')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Brandon Ingram')]['PlusMinusOffCourt'].mean()
    },
    'Kyle Kuzma': {
        'on_court': merged_df[(merged_df['Player'] == 'Kyle Kuzma')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Kyle Kuzma')]['PlusMinusOffCourt'].mean()
    },
    'Rajon Rondo': {
        'on_court': merged_df[(merged_df['Player'] == 'Rajon Rondo')]['PlusMinusOnCourt'].mean(),
        'off_court': merged_df[(merged_df['Player'] == 'Rajon Rondo')]['PlusMinusOffCourt'].mean()
    }
}


print("Average Plus-Minus With Star On and Off the Court:")
for player, stats in avg_plus_minus_updated.items():
    print(f"\n{player}:")
    print(f" On Court: {stats['on_court']:.2f}")
    print(f" Off Court: {stats['off_court']:.2f}")

          GameID          Player  PlusMinusOnCourt  PlusMinusOffCourt
0  ="0021800500"      Lonzo Ball              92.0                7.0
1  ="0021800500"  Brandon Ingram              78.0               21.0
2  ="0021800500"      Kyle Kuzma              95.0                4.0
3  ="0021800500"       Josh Hart              25.0               74.0
4  ="0021800500"     Rajon Rondo               4.0               95.0
Average Plus-Minus With Star On and Off the Court:

Lonzo Ball:
 On Court: 68.47
 Off Court: 97.47

Michael Beasley:
 On Court: 1.55
 Off Court: 164.40

Reggie Bullock:
 On Court: 32.31
 Off Court: 133.64

Kentavious Caldwell-Pope:
 On Court: 58.02
 Off Court: 107.93

Alex Caruso:
 On Court: 12.18
 Off Court: 153.76

Josh Hart:
 On Court: 62.64
 Off Court: 103.31

Brandon Ingram:
 On Court: 77.91
 Off Court: 88.04

Kyle Kuzma:
 On Court: 124.35
 Off Court: 41.60

Rajon Rondo:
 On Court: 52.18
 Off Court: 113.76
